## Import Libraries

In [9]:
import numpy as np
import pandas as pd
import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

np.random.seed(13)
print(sklearn.__version__)

1.0.2


## Import data

In [2]:
df = pd.read_csv("../data/JFP Credential Data - ProQuest v2.csv")

## Extract Credentials

In [3]:
df["Credentials"] = (df
                     .Authors
                     .apply(lambda x: re.findall(r"(CFP|PhD|CFA|JD|CPA|CLU|ChFC|LLM|AIF|CRC)", str(x)))
                     .apply(lambda x: np.nan if len(x) == 0 else x)
                     )

## Group Credentials into target groups

In [4]:
def target_grouping(x):
    if type(x) != list:
        return np.nan
    elif set(x) == {"PhD"}:
        return "Academic"
    elif "PhD" in x and len(set(x)) == 2:
        return "Both"
    else:
        return "Practitioner"

df["target_grouping"] = df.Credentials.apply(target_grouping)

## Drop nulls

In [5]:
df = df.dropna().reset_index(drop=True)

In [6]:
df.head()

,Journal,Title,Abstract,Authors,Tax_Title_Count,Tax_Title_Binary,Tax_Abstract_Count,Tax_Abstract_Binary,Tax_Total_Count,Tax_Total_Binary,...,LLM_Count,LLM_Binary,AIF_Count,AIF_Binary,CRC_Count,CRC_Binary,pubdate,year,Credentials,target_grouping
0,JFP,Tax Considerations for Relatively Wealthy Hous...,Reichenstein talks about households whose inco...,"Reichenstein, William, PhD CFA",1,1,2,1,3,1,...,0,0,0,0,0,0,21-Dec,2021,"[PhD, CFA]",Both
1,JFP,Using Scaffolding Learning Theory as a Framewo...,Financial professionals face client resistance...,"Sterbenz, Elizabeth, LMFT;Ross, Dylan L, CFP® ...",0,0,0,0,0,0,...,0,0,0,0,0,0,21-Dec,2021,"[CFP, PhD, PhD, CFP]",Both
2,JFP,How to Have Successful Client Conversations Ab...,Heye discusses how to have successful client c...,"Heye, Chris, PhD",0,0,0,0,0,0,...,0,0,0,0,0,0,21-Dec,2021,[PhD],Academic
3,JFP,To Reduce the Risk of Retirement Portfolio Exh...,Financial planners have long regarded diversif...,"Walker, Philip;Sacks, Barry H, PhD JD;Sacks, S...",0,0,0,0,0,0,...,0,0,0,0,0,0,21-Dec,2021,"[PhD, JD, PhD]",Both
4,JFP,My Career Path: Featuring TJ Burkett,Burkett shares his career path. He joined the ...,"Burkett, T J, CFP®",0,0,0,0,0,0,...,0,0,0,0,0,0,21-Dec,2021,[CFP],Practitioner


## Split Train and Target

In [7]:
X = df.Abstract
y = df.target_grouping

## Encode Target

In [8]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

## Create Pipeline

In [28]:
pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("scaler", StandardScaler(with_mean=False))
])

models = [LogisticRegression(multi_class="ovr"), MultinomialNB(), DecisionTreeClassifier(), RandomForestClassifier()]

pipelines = [Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("scaler", StandardScaler(with_mean=False)),
    ("classifier", model)
    ]) for model in models]

In [29]:
pipelines

[Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('scaler', StandardScaler(with_mean=False)),
                 ('classifier', LogisticRegression(multi_class='ovr'))]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('scaler', StandardScaler(with_mean=False)),
                 ('classifier', MultinomialNB())]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('scaler', StandardScaler(with_mean=False)),
                 ('classifier', DecisionTreeClassifier())]),
 Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                 ('scaler', StandardScaler(with_mean=False)),
                 ('classifier', RandomForestClassifier())])]

## Cross Validation

In [33]:
results = [cross_validate(pipeline, X, y, scoring=["precision_weighted", "recall_weighted"], cv=5, n_jobs=-1) for pipeline in pipelines]

/Users/andy/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andy/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andy/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/andy/opt/anaconda3/lib/python3.7/site-packages/sklearn

In [37]:
result_df = pd.DataFrame({
    "model": models,
    "avg_precision_weighted": [result["test_precision_weighted"].mean() for result in results],
    "avg_recall_weighted": [result["test_recall_weighted"].mean() for result in results]
})

In [38]:
result_df

,model,avg_precision_weighted,avg_recall_weighted
0,LogisticRegression(multi_class='ovr'),0.684750,0.755714
1,MultinomialNB(),0.685510,0.705839
2,DecisionTreeClassifier(),0.629355,0.618748
3,RandomForestClassifier(),0.586084,0.748101


## Save Results

In [39]:
result_df.to_csv("../results/baseline_model_results.csv", index=False)